# CREATE DATASET

In [ ]:
import csv
from collections import Counter
import glob
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# load annotations
crops = glob.glob('/root/data/erko/thomas_detection/fish_crop_lice_couting/*/*.png')

In [ ]:
dataset = []
for crop in crops:
    ts, xmin, ymin = os.path.basename(crop).split('_')[0:3]
    img_path = '/root/data/erko/labeled_frames/' + ts
    if not os.path.isfile(img_path):
        continue
    height, width, _ = np.array(Image.open(crop)).shape
    dataset.append([img_path, int(ymin), int(xmin), int(ymin)+width, int(xmin)+height, '1'])


In [ ]:
# dataset

In [ ]:
# plt.imshow(np.array(Image.open('/root/data/erko/labeled_frames/1533133023365'))[0:296, 166:1025 ])

In [ ]:
dataset = sorted(dataset, key=lambda k:k[0])

In [ ]:
with open('/root/data/erko/thomas_detection/train.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for ts in dataset[:675]:
        writer.writerow(ts)

In [ ]:
with open('/root/data/erko/thomas_detection/val.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for ts in dataset[675:]:
        writer.writerow(ts)

# DETECTION

In [1]:
!nvidia-smi

Thu Aug 23 23:35:21 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 37%   63C    P0    63W / 250W |      0MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 23%   42C    P8     9W / 250W |  10787MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
import os
import csv
import sys
import tensorflow as tf
import keras
import keras.preprocessing.image
import json
import random
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import math

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.callbacks import Callback
# keras_retinanet imports
from keras_retinanet import models
from keras_retinanet import losses
from keras_retinanet import layers
from keras_retinanet.models.retinanet import retinanet_bbox

from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.callbacks.eval import Evaluate
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
# sys.path.append('/root/amol/product_detection/keras-retinanet/keras_retinanet/preprocessing/')
# from csv_generator import CSVGenerator
# from ..models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet
# from keras_retinanet.models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet


from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.utils.keras_version import check_keras_version
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from eval_modified import evaluate

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [5]:
annotations = '/root/data/erko/thomas_detection/train.csv'
classes = '/root/data/erko/thomas_detection/classid.csv'
batch_size = 4
val_annotations = '/root/data/erko/thomas_detection/val.csv'

## generators

In [6]:
def create_generators(annotations,classes,batch_size=1,val_annotations=''):
    # create random transform generator for augmenting training data
    transform_generator = random_transform_generator(flip_x_chance=0.5)
    
    transform_generator = random_transform_generator(
    min_rotation=-0.2,
    max_rotation=0.2,
    min_translation=(-0.3, -0.3),
    max_translation=(0.3, 0.3),
    min_shear=-0.3,
    max_shear=0.3,
    min_scaling=(0.5, 0.5),
    max_scaling=(1.3, 1.3),
    flip_x_chance=0,
    flip_y_chance=0.5)
    
    
    train_generator = CSVGenerator(
        annotations,
        classes,
        transform_generator=transform_generator,
        batch_size=batch_size,
        image_min_side=800,
        image_max_side=1500
    )


    if val_annotations:
        validation_generator = CSVGenerator(
            val_annotations,
            classes,
            batch_size=batch_size,
            image_min_side=800,
            image_max_side=1500
        )
    else:
        validation_generator = None
    

    return train_generator, validation_generator

In [7]:
train_generator, validation_generator = create_generators(annotations, classes, batch_size, val_annotations)

In [8]:
print(train_generator.size())

261


## model

In [9]:
def model_with_weights(model, weights, skip_mismatch):
    if weights is not None:
        model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
    return model

In [10]:
def create_models(backbone_retinanet, num_classes, weights, multi_gpu=0, freeze_backbone=False):
    modifier = freeze_model if freeze_backbone else None

    # Keras recommends initialising a multi-gpu model on the CPU to ease weight sharing, and to prevent OOM errors.
    # optionally wrap in a parallel model
    if multi_gpu > 1:
        with tf.device('/cpu:0'):
            model = model_with_weights(backbone_retinanet(num_classes, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = multi_gpu_model(model, gpus=multi_gpu)
    else:
        model          = model_with_weights(backbone_retinanet(num_classes, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = model

    # make prediction model
    prediction_model = retinanet_bbox(model=model)

    # compile model
    training_model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )

    return model, training_model, prediction_model

In [11]:
backbone = models.backbone('resnet50')
weights = backbone.download_imagenet()

In [12]:
model, training_model, prediction_model = create_models(
    backbone_retinanet=backbone.retinanet,
    num_classes=train_generator.num_classes(),
    weights=weights
)

## train

In [13]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 1e-5
    drop = 0.5
    epochs_drop = 20.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lr_scheduler = LearningRateScheduler(step_decay)

In [14]:
# save model
filepath = os.path.join('/root/data/models/erko/detection/', 'model_{epoch:02d}.h5')
checkpoint = ModelCheckpoint(filepath)

In [15]:
# create history callback
class SaveHistory(Callback):
    
    def __init__(self, json_path):
        self.json_path = json_path
    
    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        with open(self.json_path, 'w') as f:
            json.dump(self.history, f)
        
saveh = SaveHistory('./history.json')

In [ ]:
# start training
history = training_model.fit_generator(
        generator=train_generator,
        steps_per_epoch=train_generator.size()//batch_size,
        epochs=40,
        verbose=1,
        validation_data= validation_generator,
        validation_steps= validation_generator.size() // batch_size,
        callbacks=[lr_scheduler, saveh, checkpoint]
    )

Epoch 1/40
65/65 [==============================] - 106s 2s/step - loss: 3.1132 - regression_loss: 2.5282 - classification_loss: 0.5850 - val_loss: 2.8501 - val_regression_loss: 2.3576 - val_classification_loss: 0.4925
Epoch 2/40
65/65 [==============================] - 96s 1s/step - loss: 2.9295 - regression_loss: 2.4003 - classification_loss: 0.5292 - val_loss: 2.7058 - val_regression_loss: 2.1777 - val_classification_loss: 0.5281
Epoch 3/40
65/65 [==============================] - 94s 1s/step - loss: 2.7663 - regression_loss: 2.2703 - classification_loss: 0.4959 - val_loss: 2.4214 - val_regression_loss: 1.9968 - val_classification_loss: 0.4246
Epoch 4/40
65/65 [==============================] - 98s 2s/step - loss: 2.5954 - regression_loss: 2.1282 - classification_loss: 0.4672 - val_loss: 2.3818 - val_regression_loss: 1.9756 - val_classification_loss: 0.4062
Epoch 5/40
65/65 [==============================] - 98s 2s/step - loss: 2.4794 - regression_loss: 2.0286 - classification_loss:

In [ ]:
rec, prec, average_precisions = evaluate(validation_generator, prediction_model, score_threshold=0.5)

In [ ]:
print(rec, prec)